<img src='https://encrypted-tbn0.gstatic.com/images?q=tbn%3AANd9GcQ-VfNtOyJbsaxu43Kztf_cv1mgBG6ZIQZEVw&usqp=CAU'>

# Procesamiento de Lenguaje Natural

## Taller #9: Modelado de temas
`Fecha de entrega: Mayo 6, 2021. (Antes del inicio de la próxima clase).`

`Modo de entrega: Subir link de GitHub al aula virtual.`

In [33]:
import pyLDAvis as p
print(p.__version__)

3.3.1


pip install -- upgrade pyLDAvis

In [1]:
import re
import pandas as pd 
from pprint import pprint

from nltk.corpus import stopwords
stopwords = stopwords.words('spanish')

import pyLDAvis.gensim_models
from gensim.models import LdaModel
from gensim.corpora import Dictionary

import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)

/opt/anaconda3/lib/python3.7/site-packages/scipy/sparse/sparsetools.py:21: DeprecationWarning: `scipy.sparse.sparsetools` is deprecated!
scipy.sparse.sparsetools is a private module for scipy.sparse, and should not be used.
  _deprecated()


In [2]:
# Cargar datos
path = '../../archivos/reviews_vidjew_es.csv'
data = pd.read_csv(path)
data.head()

,review_id,product_id,reviewer_id,stars,review_body,review_title,language,product_category
0,es_0825565,product_es_0370490,reviewer_es_0174781,3,"Buen. Buena calidad, y buena presentación.",Contenta,es,jewelry
1,es_0227934,product_es_0354224,reviewer_es_0411613,3,"Un producto a perfecto, para salir de casa con...",Versatilidad,es,video_games
2,es_0468601,product_es_0665460,reviewer_es_0348315,1,No funciona con Nintendo Switch. No hay forma ...,Decepción absoluta,es,video_games
3,es_0814494,product_es_0692692,reviewer_es_0951508,5,"Recomendado, los utilizo para pc y no me dan n...",Auriculares Pecham ps4,es,video_games
4,es_0206329,product_es_0728826,reviewer_es_0493255,4,El cable funciona bien podria ser un poco mas ...,Perfecto,es,video_games


###  `[12 pts]` Punto 1: Hacer pre-procesamiento del texto

In [3]:
def pre_procesado(texto):
    texto = texto.lower()
    texto = re.sub(r"[\W\d_]+", " ", texto)
    texto = [palabra for palabra in texto.split() if palabra not in stopwords]
    return texto

data['pp'] = data['review_body'].apply(lambda texto: pre_procesado(texto))

data.head()

,review_id,product_id,reviewer_id,stars,review_body,review_title,language,product_category,pp
0,es_0825565,product_es_0370490,reviewer_es_0174781,3,"Buen. Buena calidad, y buena presentación.",Contenta,es,jewelry,"[buen, buena, calidad, buena, presentación]"
1,es_0227934,product_es_0354224,reviewer_es_0411613,3,"Un producto a perfecto, para salir de casa con...",Versatilidad,es,video_games,"[producto, perfecto, salir, casa, nintendo, sw..."
2,es_0468601,product_es_0665460,reviewer_es_0348315,1,No funciona con Nintendo Switch. No hay forma ...,Decepción absoluta,es,video_games,"[funciona, nintendo, switch, forma, emparejarl..."
3,es_0814494,product_es_0692692,reviewer_es_0951508,5,"Recomendado, los utilizo para pc y no me dan n...",Auriculares Pecham ps4,es,video_games,"[recomendado, utilizo, pc, dan, ningún, proble..."
4,es_0206329,product_es_0728826,reviewer_es_0493255,4,El cable funciona bien podria ser un poco mas ...,Perfecto,es,video_games,"[cable, funciona, bien, podria, ser, mas, larg..."


###  `[13 pts]` Punto 2: Modelo de LDA

In [4]:
# Crear una representación de los documentos en forma de diccionario
dictionary = Dictionary(data['pp'].values)

# Filtrar palabras muy frecuentes o infrecuentes
dictionary.filter_extremes(no_below=5, no_above=0.5)

corpus = [dictionary.doc2bow(text) for text in data['pp'].values]

#Train the topic model
model = LdaModel(corpus=corpus, id2word=dictionary, num_topics=6, passes=50)

###  `[25 pts]` Punto 3: Visualización de LDA

In [5]:
lda_display = pyLDAvis.gensim_models.prepare(model, corpus, dictionary, sort_topics=False)
pyLDAvis.display(lda_display)
# pyLDAvis.save_html(lda_display, 'lda.html')

# Más

In [6]:
model.print_topics(num_words=10)

[(0,
  '0.033*"juego" + 0.033*"llegado" + 0.032*"producto" + 0.017*"bonitos" + 0.015*"nintendo" + 0.014*"hijo" + 0.014*"pedido" + 0.013*"switch" + 0.012*"pequeña" + 0.011*"entrega"'),
 (1,
  '0.024*"juego" + 0.020*"buena" + 0.020*"si" + 0.019*"buen" + 0.017*"cumple" + 0.017*"precio" + 0.015*"compra" + 0.015*"gran" + 0.013*"compré" + 0.013*"bueno"'),
 (2,
  '0.066*"bien" + 0.023*"bonito" + 0.021*"precio" + 0.019*"cadena" + 0.019*"plata" + 0.016*"calidad" + 0.016*"queda" + 0.015*"regalo" + 0.015*"tamaño" + 0.015*"colgante"'),
 (3,
  '0.083*"calidad" + 0.037*"precio" + 0.034*"buena" + 0.024*"pulsera" + 0.022*"bonito" + 0.021*"foto" + 0.021*"perfecto" + 0.019*"mas" + 0.016*"tiempo" + 0.015*"día"'),
 (4,
  '0.030*"producto" + 0.029*"bien" + 0.025*"mando" + 0.020*"mas" + 0.016*"amazon" + 0.015*"llego" + 0.014*"cable" + 0.013*"solo" + 0.012*"consola" + 0.012*"comprar"'),
 (5,
  '0.042*"si" + 0.036*"juego" + 0.019*"bien" + 0.017*"ps" + 0.017*"original" + 0.016*"perfecto" + 0.014*"tiempo" + 0.0

In [7]:
def get_doc_top_n(text_processed, n):
    d = dictionary.doc2bow(text_processed)
    topics = dict(model.get_document_topics(d))
    try:
        return topics[n]
    except:
        return None

In [15]:
for t in range(0,6):
    top_name = f"topic_{t}"
    data[top_name] = data['pp'].apply(lambda doc: get_doc_top_n(doc, t))

In [16]:
data

,review_id,product_id,reviewer_id,stars,review_body,review_title,language,product_category,pp,topic_0,topic_1,topic_2,topic_3,topic_4,topic_5,topic_6
0,es_0825565,product_es_0370490,reviewer_es_0174781,3,"Buen. Buena calidad, y buena presentación.",Contenta,es,jewelry,"[buen, buena, calidad, buena, presentación]",0.027815,0.027970,0.027810,0.860755,0.027834,0.027810,None
1,es_0227934,product_es_0354224,reviewer_es_0411613,3,"Un producto a perfecto, para salir de casa con...",Versatilidad,es,video_games,"[producto, perfecto, salir, casa, nintendo, sw...",0.564640,0.027841,0.027797,0.323777,0.028077,0.027912,None
2,es_0468601,product_es_0665460,reviewer_es_0348315,1,No funciona con Nintendo Switch. No hay forma ...,Decepción absoluta,es,video_games,"[funciona, nintendo, switch, forma, emparejarl...",0.689229,0.024055,0.023904,0.023863,0.023880,0.215333,None
3,es_0814494,product_es_0692692,reviewer_es_0951508,5,"Recomendado, los utilizo para pc y no me dan n...",Auriculares Pecham ps4,es,video_games,"[recomendado, utilizo, pc, dan, ningún, proble...",0.276652,0.020894,0.021099,0.020865,0.639657,0.020903,None
4,es_0206329,product_es_0728826,reviewer_es_0493255,4,El cable funciona bien podria ser un poco mas ...,Perfecto,es,video_games,"[cable, funciona, bien, podria, ser, mas, larg...",0.013954,0.013926,0.013930,0.014135,0.930117,0.013937,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,es_0427672,product_es_0899883,reviewer_es_0474735,2,En la foto parece que la cadena sea más gruesa...,Cadena muy fina,es,jewelry,"[foto, parece, cadena, gruesa, fina, precio, p...",0.018750,0.018695,0.906263,0.018893,0.018771,0.018645,None
996,es_0059540,product_es_0702564,reviewer_es_0754753,3,"Bien por su precio, la cadena demasiado fina.",E,es,jewelry,"[bien, precio, cadena, demasiado, fina]",0.027888,0.027833,0.860696,0.027938,0.027818,0.027829,None
997,es_0303349,product_es_0678671,reviewer_es_0172313,5,Muy bien!! Ahora tengo organizados los pendien...,Bonito,es,jewelry,"[bien, ahora, organizados, pendientes]",0.042103,0.042083,0.790254,0.041668,0.042181,0.041715,None
998,es_0659961,product_es_0294943,reviewer_es_0554554,4,"Las tapas para los joystics no están nada mal,...",No está mal pero sólo he probado las fundas.,es,video_games,"[tapas, joystics, mal, ayudan, montón, joystic...",0.213187,0.012903,0.012896,0.012865,0.012888,0.735298,None


In [32]:
d = dictionary.doc2bow(["juego", "perfecto", "jugar", "rápido"])
topics = model.get_document_topics(d, per_word_topics=True)
topics

([(0, 0.033643615),
  (1, 0.03359808),
  (2, 0.033345647),
  (3, 0.03355201),
  (4, 0.03338128),
  (5, 0.83247936)],
 [(6, [5]), (11, [5]), (56, [5]), (121, [5])],
 [(6, [(5, 0.9990089)]),
  (11, [(5, 0.99887645)]),
  (56, [(5, 0.999138)]),
  (121, [(5, 0.9986993)])])

In [23]:
d = dictionary.doc2bow(["arete", "calidad", "pulsera"])
topics = model.get_document_topics(d, per_word_topics=True)
topics

([(0, 0.055616032),
  (1, 0.055606958),
  (2, 0.05567204),
  (3, 0.72196126),
  (4, 0.055585675),
  (5, 0.05555805)],
 [(2, [3]), (128, [3])],
 [(2, [(3, 0.99954295)]), (128, [(3, 0.9996729)])])

In [25]:
dictionary.token2id['juego']

56